In [1]:
# downloading Selenium libraries 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# downloading BeautifulSoup
from bs4 import BeautifulSoup

# downloading Numpy & Pandas
import pandas as pd
import numpy as np

# downloading sleep function 
from time import sleep
import time

# downloading additional libraries
from requests import get
import requests, openpyxl
import re

from tqdm.notebook import tqdm

# downloading warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# downloading fred & setting fred api key
import fred
fred.key('8c3b945500069081b94040df2da12df7')

In [2]:
# setting Selenium options 
options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
# setting website that should be scraped 
driver.get("https://fred.stlouisfed.org/releases/calendar")

In [3]:
def get_table():
    
    """
    get_table function retrieves tables from the website, transforms it into a dataframe and 
    modifies it in order to obtain the most important information in a transparent way
    """
        
    html=driver.page_source
    html = html.replace("N/A","Unspecified")
    
    soup=BeautifulSoup(html,'html.parser')
    
    # downloading the table from the website
    div=soup.find_all('table', attrs={'class':'table table-condensed table-standard-theme'})
    table=pd.read_html(str(div))
    
    # transforming the table to string
    ans = np.array(table)
    ans = (ans[0])
    
    # saving array as a dataframe 
    df = pd.DataFrame(ans, columns = ['Hour','Variable'])
    
    # saving new variable
    globals()['last_day_of_the_week'] = df['Variable'].iloc[0]

    # editing the dataframe
    df['Hour'].fillna(method='pad', inplace=True)
    df['Date'] = df['Variable'].iloc[0]
    df = df.iloc[1: , :]
    dfx = df['Date'].str.split(' ', expand=True)
    df = df.drop('Date', axis=1)
    df = pd.concat([dfx, df.reindex(dfx.index)], axis=1)
    df[2] = df[2].replace(',','', regex=True)
    df.rename(columns = {1:'Month', 2:'Day', 3:'Year'}, inplace = True)
    get_table.df = df
    
    # saving the weekday of the table to be able to delte it in next step
    weekday = df[0].iloc[0]
    df = df.drop(0, axis=1)
    
    subcategories = soup.find("table", {"class": "table table-condensed table-standard-theme"})
    test = []

    for variable in df['Variable']:
        for subcategory in subcategories.find_all("a", href=re.compile("rid"), text=re.compile(variable)):
            test.append(subcategory.get('href'))
            
    dfx = pd.DataFrame(test)
    dfx = dfx.rename(columns={0: "Rid"})
    dfx = dfx["Rid"].str.replace("?", '', regex=True)
    dfx = dfx.str.replace("/releaserid=", '', regex=True)
    dfx.index = np.arange(1, len(dfx) + 1)
    dfx = dfx.replace('/release?rid=',' ', regex=True)
    dfx = dfx.astype(str).astype(int)
    dfnew = pd.concat([df, dfx], axis=1, join="inner")
        
    # saving results as a global df 
    globals()['day_1'] = dfnew
    
    # writed to confirm the execution of the function 
    print(weekday + ", " + df['Day'].iloc[0] + " " + df['Month'].iloc[0] + " downloaded")

In [4]:
def download_data(name):
    
    """
    a function that combines a dataframe from individual days into one large dataframe for a whole week
    """
    
    # merging all df's to week_df
    week_df = pd.concat([day_1])
    

    week_df.rename(columns = {4:'Updated'}, inplace = True)
    week_df = week_df.reset_index()
    globals()[name] = week_df

In [5]:
def wait():
    
    """
    feature designed for delays allowing the full page loading
    """
    
    WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH, "//table[@class='fc-border-separate']/tbody/tr/td")));
    sleep(3)

In [6]:
# click on the day you want to scrape
get_table()

# saving data as week_1
download_data('week_1')

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=108.0.5359.94)
Stacktrace:
0   chromedriver                        0x0000000102d25f38 chromedriver + 4910904
1   chromedriver                        0x0000000102ca5a03 chromedriver + 4385283
2   chromedriver                        0x00000001028ea747 chromedriver + 472903
3   chromedriver                        0x00000001028c0ff5 chromedriver + 303093
4   chromedriver                        0x0000000102959b0f chromedriver + 928527
5   chromedriver                        0x000000010296f763 chromedriver + 1017699
6   chromedriver                        0x0000000102954ee3 chromedriver + 909027
7   chromedriver                        0x000000010291f30c chromedriver + 688908
8   chromedriver                        0x000000010292088e chromedriver + 694414
9   chromedriver                        0x0000000102cf31de chromedriver + 4702686
10  chromedriver   

In [ ]:
# creating empty dataframes that will be filled later
data_links = pd.DataFrame()
data_you_need = pd.DataFrame()

In [ ]:
def get_rid():
    
    """
    function downloading variables inside rid groups
    """

    global data_you_need
    
    html=driver.page_source
    html = html.replace("N/A","Unspecified")

    soup=BeautifulSoup(html,'html.parser')
    
    subcategories = soup.find("table", {"id": "series-pager"})
    test2 = []
    test3 = []
    test4 = []

    for link in subcategories.find_all("a", href=re.compile("/series/"), attrs={'class':'pager-series-attributes-gtm'}):
        test2.append(link.get('href'))
        
    for link2 in subcategories.find_all("span", attrs={'class':'series-meta-dates'}):
        test3.append(link2.get_text('span'))
        
    for link3 in subcategories.find_all("span", attrs={'class':'attributes'}):
        test4.append(link3.get_text('span'))
        
    dfx2 = pd.DataFrame(test2)
    dfx2 = dfx2.rename(columns={0: "VariableID"})
    dfx2 = dfx2["VariableID"].str.replace("/series/", '', regex=True)
    dfx2.index = np.arange(1, len(dfx2) + 1)
    dfx2 = pd.DataFrame(dfx2)
    dfx2['Rid'] = rid
    
    dfx3 = pd.DataFrame(test3)
    dfx3 = dfx3.rename(columns={0: "Last_updated"})
    dfx3.index = np.arange(1, len(dfx3) + 1)
    dfx3 = pd.DataFrame(dfx3)
    
    dfx4 = pd.DataFrame(test4)
    dfx4 = dfx4.rename(columns={0: "Attributes"})
    dfx4.index = np.arange(1, len(dfx4) + 1)
    dfx4 = pd.DataFrame(dfx4)
    #dfx4 = dfx4['Attributes'].str.split(',', expand=True)
    
    #dfx4 = dfx4.rename(columns={0: "Attributes",1: "test1",2: "test2",3:"test3"})
    
    dfx3 = dfx3['Last_updated'].str.split('(', expand=True)
    dfx3 = dfx3.rename(columns={0: "variable_date_range"})
    dfx3 = dfx3.rename(columns={1: "last_updated"})
    
    dfx3['last_updated'] = dfx3['last_updated'].str.replace(')', '')    
    dfx3['last_updated'] = dfx3['last_updated'].str.replace('\n', '')
    dfx3['variable_date_range'] = dfx3['variable_date_range'].str.replace('\n', '')
    
    dfx3 = dfx3.join(dfx4)
    data = dfx2.join(dfx3)
        
    data_you_need = data_you_need.append(data,ignore_index=True)

In [ ]:
for rid in tqdm(week_1["Rid"]):
    URL = "https://fred.stlouisfed.org/release?rid=" + str(rid)
    
    driver.get(URL)
    
    while True:
        try:
            next_page = driver.find_element(By.XPATH, "//a[@title='next page']")
            next_page.click()
            get_rid();
        except:
            get_rid()
            break

In [10]:
# merging data
df = pd.merge(week_1, data_you_need, on='Rid', how='outer')

In [11]:
# deleting duplicates
df = df.drop_duplicates(['Rid','VariableID'], keep='last')

In [12]:
# deleting all variables except monthly 
# df = df[df['Attributes'].str.contains("Monthly", na=False)]
df = df.reset_index(drop=True)
df = df.drop(['Attributes'], axis=1)

In [13]:
df

,index,Month,Day,Year,Hour,Variable,Rid,VariableID,variable_date_range,last_updated
0,8,December,07,2022,8:00 am,Overnight Bank Funding Rate Data,349,ASMD,Jan 2000 to Jul 2022,Nov 9
1,12,December,07,2022,12:00 pm,Spot Prices,86,DTBSPCKCLCNM,Jan 2001 to Dec 2006,May 4
2,12,December,07,2022,12:00 pm,Spot Prices,86,DTBSPCKNDNM,Jan 2001 to Nov 2022,5 days ago
3,12,December,07,2022,12:00 pm,Spot Prices,86,DTBSPCKFDUNM,Jan 2001 to Nov 2022,5 days ago
4,12,December,07,2022,12:00 pm,Spot Prices,86,DTBSPCKNOWW,2001-01-03 to 2022-11-30,5 days ago
...,...,...,...,...,...,...,...,...,...,...
136,21,December,07,2022,Unspecified,Recession Indicators Series,279,EMVCOMPPOL,Jan 1985 to Oct 2022,Nov 7
137,21,December,07,2022,Unspecified,Recession Indicators Series,279,EMVMACROLABORMKT,Jan 1985 to Oct 2022,Nov 7
138,21,December,07,2022,Unspecified,Recession Indicators Series,279,EMVIPPOL,Jan 1985 to Oct 2022,Nov 7
139,21,December,07,2022,Unspecified,Recession Indicators Series,279,EMVOTHERREG,Jan 1985 to Oct 2022,Nov 7


In [14]:
# test
fred.series('NIKKEI225')

{'realtime_start': '2022-12-07',
 'realtime_end': '2022-12-07',
 'seriess': [{'id': 'NIKKEI225',
   'realtime_start': '2022-12-07',
   'realtime_end': '2022-12-07',
   'title': 'Nikkei Stock Average, Nikkei 225',
   'observation_start': '1949-05-16',
   'observation_end': '2022-12-06',
   'frequency': 'Daily, Close',
   'frequency_short': 'D',
   'units': 'Index',
   'units_short': 'Index',
   'seasonal_adjustment': 'Not Seasonally Adjusted',
   'seasonal_adjustment_short': 'NSA',
   'last_updated': '2022-12-06 07:02:04-06',
   'popularity': 57,
   'notes': 'The observations for the Nikkei Stock Average, Nikkei 225 represent the daily index value at market close.\n\nNikkei 225 is the major stock market index comprising of 225 highly liquid stocks of the Tokyo Stock Exchange (TSE).\n\nFor in depth information, visit here (http://indexes.nikkei.co.jp/nkave/archives/faq/faq_nikkei_stock_average_en.pdf).\n\nCopyright, 2016, Nikkei Inc. Reprinted with permission.\n\nDownloading the data for

In [15]:
title = []
frequency = []
last_updated = []


check = -1
outof = len(df.index) - 1


for VariableID in df['VariableID']:
    global title, frequency, last_updated
    
    check += 1
    print(check, "/", outof)
    
    time.sleep(1)
    
    if fred.series(VariableID)['seriess']:
        dictionary = fred.series(VariableID)['seriess']
        title_ = dictionary[0]['title']
        frequency_ = dictionary[0]['frequency']
        last_updated_ = dictionary[0]['last_updated']
    else:
        title_ = "did not succeed to get data"
        frequency_ = "did not succeed to get data" 
        last_updated_ = "did not succeed to get data"
        
    title.append(str(title_))
    frequency.append(str(frequency_))
    last_updated.append(str(last_updated_))

0 / 140
1 / 140
2 / 140
3 / 140
4 / 140
5 / 140
6 / 140
7 / 140
8 / 140
9 / 140
10 / 140
11 / 140
12 / 140
13 / 140
14 / 140
15 / 140
16 / 140
17 / 140
18 / 140
19 / 140
20 / 140
21 / 140
22 / 140
23 / 140
24 / 140
25 / 140
26 / 140
27 / 140
28 / 140
29 / 140
30 / 140
31 / 140
32 / 140
33 / 140
34 / 140
35 / 140
36 / 140
37 / 140
38 / 140
39 / 140
40 / 140
41 / 140
42 / 140
43 / 140
44 / 140
45 / 140
46 / 140
47 / 140
48 / 140
49 / 140
50 / 140
51 / 140
52 / 140
53 / 140
54 / 140
55 / 140
56 / 140
57 / 140
58 / 140
59 / 140
60 / 140
61 / 140
62 / 140
63 / 140
64 / 140
65 / 140
66 / 140
67 / 140
68 / 140
69 / 140
70 / 140
71 / 140
72 / 140
73 / 140
74 / 140
75 / 140
76 / 140
77 / 140
78 / 140
79 / 140
80 / 140
81 / 140
82 / 140
83 / 140
84 / 140
85 / 140
86 / 140
87 / 140
88 / 140
89 / 140
90 / 140
91 / 140
92 / 140
93 / 140
94 / 140
95 / 140
96 / 140
97 / 140
98 / 140
99 / 140
100 / 140
101 / 140
102 / 140
103 / 140
104 / 140
105 / 140
106 / 140
107 / 140
108 / 140
109 / 140
110 / 140


In [16]:
title_df = pd.DataFrame(title)
frequency_df = pd.DataFrame(frequency)
last_updated_df = pd.DataFrame(last_updated)

In [17]:
merged1 = pd.concat([title_df, frequency_df], keys=['title_df', 'frequency_df'], axis=1)#
merged2 = pd.concat([merged1, last_updated_df], axis=1)
merged3 = pd.concat([df, merged2], axis=1)
df = merged3

In [18]:
df.rename({"'title_df)": 'Title', "('frequency_df', 0)": 'Freqency', int(0):'Last_updated', 'Month':'Month_name'}, axis=1, inplace=True)

In [19]:
df['Month'] = pd.to_datetime(df.Month_name, format='%B').dt.month.astype(str)

In [20]:
df['Date_of_planned_update']=pd.to_datetime(df[['Year','Month','Day']])

In [21]:
df.dtypes

index                              int64
Month_name                        object
Day                               object
Year                              object
Hour                              object
Variable                          object
Rid                                int32
VariableID                        object
variable_date_range               object
last_updated                      object
(title_df, 0)                     object
(frequency_df, 0)                 object
Last_updated                      object
Month                             object
Date_of_planned_update    datetime64[ns]
dtype: object

In [22]:
df['Date_of_last_update'] = pd.to_datetime(df['Last_updated'])
df['Date_of_planned_update'] = pd.to_datetime(df['Date_of_planned_update'])

In [23]:
df.dtypes

index                              int64
Month_name                        object
Day                               object
Year                              object
Hour                              object
Variable                          object
Rid                                int32
VariableID                        object
variable_date_range               object
last_updated                      object
(title_df, 0)                     object
(frequency_df, 0)                 object
Last_updated                      object
Month                             object
Date_of_planned_update    datetime64[ns]
Date_of_last_update               object
dtype: object

In [24]:
df.to_csv('meh.csv')